# Central Bank Policy Rates - BIS

## Python set-up

In [1]:
# system imports
from pathlib import Path
from urllib.error import HTTPError, URLError
from typing import Any
import textwrap as tw
from collections.abc import Iterable

In [2]:
# analytic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycountry

In [3]:
# local imports
from mgplot import (
    set_chart_dir,
    clear_chart_dir,
    line_plot_finalise,
    finalise_plot,
)

In [4]:
# plotting set-up
SOURCE = "Source: BIS policy rates"
LFOOTER = "Daily data.  Note: There are lags in BIS data reporting. "
plt.style.use("fivethirtyeight")
CHART_DIR = "./CHARTS/BIS/"
set_chart_dir(CHART_DIR)
clear_chart_dir()
SHOW = False

## Data capture

In [5]:
def country_code_to_name(country_code: str) -> str:
    """Convert 2-digit country codes to country names."""

    try:
        country = pycountry.countries.get(alpha_2=country_code)
        return country.name
    except AttributeError:
        if country_code == "XM":
            return "Euro Area"
        return country_code

In [6]:
def index_missing_dates(data: pd.DataFrame | pd.Series) -> pd.DataFrame | pd.Series:
    """Reindex [and sort] a Series/DataFrame to include all missing dates in the index.
    This function works for data with either a DatetimeIndex or PeriodIndex."""

    # check that the index is a DatetimeIndex or PeriodIndex
    assert isinstance(data.index, pd.DatetimeIndex) or isinstance(
        data.index, pd.PeriodIndex
    )

    function = (
        pd.period_range if isinstance(data.index, pd.PeriodIndex) else pd.date_range
    )
    index = function(start=data.index.min(), end=data.index.max(), freq="D")
    data = data.reindex(index, fill_value=np.nan)
    data = data.sort_index()
    data = data.ffill()

    return data

In [7]:
# takes a few minutes to run

def get_bis_data(start="2018-01-01") -> tuple[pd.DataFrame, str]:
    """
    Get central bank policy rates from the BIS for a select set of states.
    Arguments: start -- the start date for the data (default: 2018-01-01)
    Returns a DataFrame of daily data and a string of the latest rates.
    """

    states = sorted(
        list(
            set(
                [  # ensure unique and sorted
                    "AU",
                    "CA",
                    "GB",
                    "JP",
                    "NO",
                    "KR",
                    "NZ",
                    "SE",
                    "US",
                    "XM",
                    "AR",
                    "BR",
                    "CL",
                    "CN",
                    "CZ",
                    "DK",
                    "HK",
                    "HU",
                    "IN",
                    "ID",
                    "IL",
                    "MY",
                    "MX",
                    "PH",
                    "PL",
                    "RU",
                    "ZA",
                    "TH",
                    "TR",
                    "CO",
                    "HR",
                    "IS",
                    "MA",
                    "MK",
                    "PE",
                    "RO",
                    "SA",
                    "RS",
                    "CH",
                ]
            )
        )
    )
    box = {}
    finals = []
    for abbr in states:
        for _trys in range(2):
            state = country_code_to_name(abbr)
            # print(state)
            url = f"https://stats.bis.org/api/v2/data/dataflow/BIS/WS_CBPOL/1.0/D.{abbr}?startPeriod={start}&format=csv"
            try:
                df = pd.read_csv(url)[["TIME_PERIOD", "OBS_VALUE"]]
            except (HTTPError, URLError) as e:
                print(f"Internet Error: {state} {e}")
                continue
            s = pd.Series(
                df["OBS_VALUE"].values,
                name=state,
                dtype=float,
                index=pd.to_datetime(df["TIME_PERIOD"]),
            )
            if s.empty or s.isnull().all():
                print(f"Empty: {state}")
                continue
            s = index_missing_dates(s)  # type: ignore[assignment]
            box[state] = s
            finals.append(f"{abbr}: {s.iloc[-1]:.2f}")
            break

    data = pd.DataFrame(box)
    data.index = pd.PeriodIndex(data.index, freq="D")

    return data, ", ".join(finals)


df, latest_rates = get_bis_data()  # takes around 3 minutes of internet time
print(df.index.dtype)

period[D]


### By hand adjustments

In [8]:
# Make adjustments when BIS data is lagged


adjustments = {
    # Nation : [adjustment, date-as-period]
    "Australia": [3.85, pd.Period("2025-05-20", freq="D")],
    "New Zealand": [3.25, pd.Period("2025-05-28", freq="D")],
}


def make_adjustments(frame: pd.DataFrame, adjust_dict: dict) -> pd.DataFrame:
    """Because the BIS data is highly lagged, we may need to adjust it for late
    policy rate changes.
    Arguments: frame -- the DataFrame to adjust
               adjust_dict -- a dictionary of adjustments
    Returns the adjusted DataFrame."""

    fail_if_too_long_ago = 21  # Days

    for state, (adj, date) in adjust_dict.items():
        if date > frame.index[-1]:
            frame.loc[date, state] = adj
            frame = index_missing_dates(frame)  # type: ignore[assignment]
        else:
            how_far_back = (date - frame.index[-1]) / np.timedelta64(1, "D")
            if how_far_back > fail_if_too_long_ago:
                print(f"Failed to adjust {state} by {adj} on {date}")
                continue
            frame.loc[date, state] = adj
            frame.loc[frame.index > date, state] = np.nan
            frame[state] = frame[state].ffill()

    return frame


df = make_adjustments(df, adjustments)

## Plotting

In [9]:
COUNTER = 0


def plot_rates(dataset: pd.DataFrame, start="2018-01-01") -> None:
    """Plot the central bank policy rates."""

    widths = [1.5] * 40
    if "Australia" in dataset.columns:
        widths[dataset.columns.get_loc("Australia")] = 3  # type: ignore[index]

    if len(dataset.columns) < 10:
        fs = 10
    elif len(dataset.columns) < 20:
        fs = 8
    else:
        fs = 6

    global COUNTER
    line_plot_finalise(
        dataset[lambda x: x.index >= start],
        title="Central Bank Policy Rates",
        ylabel="Annual Policy Rate (%)",
        rfooter=SOURCE,
        lfooter=LFOOTER,
        width=widths,
        style=["-", "--", ":", "-."] * 10,
        legend={"ncols": 3, "loc": "upper left", "fontsize": fs},
        y0=True,
        zero_y=True,
        tag=f"{COUNTER}",
        show=SHOW,
    )
    COUNTER += 1


plot_rates(
    df[
        [
            "Australia",
            "Canada",
            "Euro Area",
            "New Zealand",
            "United Kingdom",
            "United States",
        ]
    ]
)
ugly = ["Argentina", "Russian Federation", "Türkiye"]
plot_rates(df[sorted([x for x in df.columns if x not in ugly])])
plot_rates(df[ugly])

In [10]:
MEAN_MEDIAN = 0.80  # proportion of non-na data points to plot mean and median


def plot_world(
    data: pd.DataFrame,
    exclusions: Iterable = tuple(),
    **kwargs: Any,
) -> None:
    """Plot Australia vs the BIS monitored mean and median."""

    # Exclude problematic BIS states
    keep = [x for x in data.columns if x not in exclusions]
    my_data = data[keep].copy()

    # plot remaining BIS states without legend label using the _ trick
    mapper = {x: f"_{x}" for x in my_data.columns}
    my_data = my_data.rename(columns=mapper)
    ax = my_data.plot(color="blue", lw=0.25, alpha=0.5)
    back = {y: x for x, y in mapper.items()}
    my_data = my_data.rename(columns=back)
    my_data["Australia"].dropna().plot(
        ax=ax, color="darkorange", lw=3, label="Australia"
    )

    # plot mean if THRESHOLD proportion of non-na data points met
    mean = my_data.mean(axis=1).where(
        my_data.notna().sum(axis=1) >= len(my_data.columns) * MEAN_MEDIAN,
        other=np.nan,
    )
    median = my_data.median(axis=1).where(
        my_data.notna().sum(axis=1) >= len(my_data.columns) * MEAN_MEDIAN,
        other=np.nan,
    )
    mean.plot(ax=ax, color="darkblue", ls="--", lw=2, label="BIS monitored mean")
    median.plot(ax=ax, color="darkred", ls=":", lw=2, label="BIS monitored median")

    # plot
    global PW_COUNTER  # yes, this is ugly
    PW_COUNTER = PW_COUNTER + 1
    finalise_plot(
        ax,
        xlabel=None,
        y0=True,
        rfooter=SOURCE,
        tag=str(PW_COUNTER),
        legend={"loc": "best", "fontsize": "xx-small"},
        **kwargs,
        show=SHOW,
    )


PW_COUNTER = 0
plot_world(
    df,
    exclusions=ugly,
    title="CB Policy Rates: Australia in World Context",
    ylabel="Annual Policy Rate (%)",
    lfooter=LFOOTER + f" Excluded: {', '.join(ugly)}.",
)

/Users/bryanpalmer/ABS/.venv/lib/python3.13/site-packages/mgplot/finalise_plot.py:317: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  fig.tight_layout(pad=1.1)


## The End

In [11]:
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-05-28 14:52:26

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.2.0

conda environment: n/a

Compiler    : Clang 20.1.0 
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

pycountry : 24.6.1
numpy     : 2.2.6
typing    : 3.10.0.0
pandas    : 2.2.3
pathlib   : 1.0.1
mgplot    : 0.0.1
matplotlib: 3.10.3

Watermark: 2.5.0

